<img src=" " height="300" width="300"> 

# <center> R для тервера и матстата. <br>  <br> 6.1 Проверка гипотез. Часть первая. </center>



Данный ноутбук является конспектом по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2017-2018). Автор ноутбука [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

---------

Ура! Мы добрались до самого главного. До проверки гипотез. Аналитики в разных компаниях проверяют их на ежедневной основе. Давайте попробуем разобраться зачем. Для начала немного кода для красивых картинок в ggplot2. Как обычно, вы не обязаны понимать этот код. Он тут нужен для красоты. 

In [3]:
library("ggplot2")  # Пакет для красивых графиков 
library("grid")     # Пакет для субплотов

# Отрегулируем размер картинок, которые будут выдаваться в нашей тетрадке
library('repr')
options(repr.plot.width=4, repr.plot.height=3)

In [4]:
# В этом блоке написаны несколько функций, которые рисуют красивые картинки для проверки гипотез.
# Можно не обращать на них внимание :) 

# Область, которую надо будет закрасить на графике 
limitRange <- function(fun, min, max) {
    function(x) {
        y <- fun(x)
        y[x < min  |  x > max] <- NA
        return(y)
    }
}

z_stat_picture <- function(z_stat, alpha =0.05, alternative = 'two-sided'){
    # Опции для размеров графика
    options(repr.plot.width=6, repr.plot.height=3)
    
    # Какие области надо закрасить (зависит от алтиернативы)
    if(alternative == 'two-sided'){
        z_crit <- qnorm(1-(alpha/2))
        dlimit_left  <- limitRange(dnorm, -Inf, -z_crit) 
        dlimit_right <- limitRange(dnorm, z_crit, Inf) 
        }
    if(alternative == 'less'){
        z_crit <- qnorm(alpha)
        dlimit_left  <- limitRange(dnorm, -Inf, z_crit)
        dlimit_right <- limitRange(dnorm, Inf, Inf)        
    }
    if(alternative == 'greater'){
        z_crit <- qnorm(1 - alpha)
        dlimit_left  <- limitRange(dnorm, -Inf, -Inf)
        dlimit_right <- limitRange(dnorm, z_crit, Inf)
        }        
      
    # Основная картинка
    p <- ggplot(data.frame(x=c(-4,4)), aes(x = x))+
       stat_function(fun=dnorm) +  # вся функция 
       stat_function(fun=dlimit_left, geom="area",  fill="blue", alpha=0.2) +
       stat_function(fun=dlimit_right, geom="area", fill="blue", alpha=0.2) +
       geom_point(aes(x=z_stat,y=0), color="red", size=2) +
       geom_vline(xintercept = z_stat, size=0.3, linetype="dashed", color="red") +
       annotate("text", label=round(z_stat,2), x= z_stat, y=0.2, parse=T, size=4, color="darkred")
    
    # Рисуем критические точки 
    if(alternative == 'two-sided'){
       p <- p + annotate("text", label=round(z_crit,2), x=z_crit+0.1, y=-0.02, parse=T, size=4) + 
                annotate("text", label=round(-z_crit,2), x=-z_crit-0.1, y=-0.02, parse=T, size=4)
        }
    if(alternative == 'less'){
       p <- p + annotate("text", label=round(z_crit,2), x=z_crit-0.1, y=-0.02, parse=T, size=4)
        }
    if(alternative == 'greater'){
       p <- p + annotate("text", label=round(z_crit,2), x=z_crit+0.1, y=-0.02, parse=T, size=4)
        }      
    
    # Немного заключительных настроек, связанных с темой и вывод на экран 
    p <- p + theme(
             axis.text.x = element_blank(),
             axis.text.y = element_blank(),
             axis.ticks = element_blank(),
             axis.title.x = element_blank(),
             axis.title.y = element_blank()
            )
    
    return(p)
}

In [5]:
# Точно такой же рисунок для t-статистики
t_stat_picture <- function(t_stat, n, alpha =0.05, alternative = 'two-sided'){
    # Опции для размеров графика
    options(repr.plot.width=6, repr.plot.height=3)
    
    # Какие области надо закрасить (зависит от алтиернативы)
    if(alternative == 'two-sided'){
        t_crit <- qt(1-(alpha/2),df=n)
        dlimit_left  <- limitRange(function(x) dt(x, df=n), -Inf, -t_crit) 
        dlimit_right <- limitRange(function(x) dt(x, df=n), t_crit, Inf) 
        }
    if(alternative == 'less'){
        t_crit <- qt(alpha,df=n)
        dlimit_left  <- limitRange(function(x) dt(x, df=n), -Inf, t_crit)
        dlimit_right <- limitRange(function(x) dt(x, df=n), Inf, Inf)        
    }
    if(alternative == 'greater'){
        t_crit <- qt(1 - alpha,df=n)
        dlimit_left  <- limitRange(function(x) dt(x, df=n), -Inf, -Inf)
        dlimit_right <- limitRange(function(x) dt(x, df=n), t_crit, Inf)
        }        
      
    # Основная картинка
    p <- ggplot(data.frame(x=c(-4, 4)), aes(x = x))+
       stat_function(fun=dt, args = list(df=n)) +  # вся функция 
       stat_function(fun=dlimit_left, geom="area",  fill="blue", alpha=0.2) +
       stat_function(fun=dlimit_right, geom="area", fill="blue", alpha=0.2) +
       geom_point(aes(x=t_stat,y=0), color="red", size=2) +
       geom_vline(xintercept = t_stat, size=0.3, linetype="dashed", color="red") +
       annotate("text", label=round(t_stat,2), x= t_stat, y=0.2, parse=T, size=4, color="darkred")
    
    # Рисуем критические точки 
    if(alternative == 'two-sided'){
       p <- p + annotate("text", label=round(t_crit,2), x=t_crit+0.1, y=-0.02, parse=T, size=4) + 
                annotate("text", label=round(-t_crit,2), x=-t_crit-0.1, y=-0.02, parse=T, size=4)
        }
    if(alternative == 'less'){
       p <- p + annotate("text", label=round(t_crit,2), x=t_crit-0.1, y=-0.02, parse=T, size=4)
        }
    if(alternative == 'greater'){
       p <- p + annotate("text", label=round(t_crit,2), x=t_crit+0.1, y=-0.02, parse=T, size=4)
        }      
    
    # Немного заключительных настроек, связанных с темой и вывод на экран 
    p <- p + theme(
             axis.text.x = element_blank(),
             axis.text.y = element_blank(),
             axis.ticks = element_blank(),
             axis.title.x = element_blank(),
             axis.title.y = element_blank()
            )
    
    return(p)
}

## Agenda 

- Как проверять гипотезы

- Точные и асимптотические критерии

- Гипотезы о долях  

- Гипотезы о средних

- Гипотезы о дисперсиях


# 1. Что такое гипотезы и зачем их проверять 

На самом деле мы можем сказать, что статистическая гипотеза это абсолютно любое утверждение, которое возникает в нашей голове, если его можно проверить с помощью данных.

#### Примеры гипотез: 

* В Австралии женщин дискриминируют на рынке труда!

* В Питере люди любят кофе больше, чем в москве!

* Посетители постоянно покупают в паре пиво и подгузники. Мы могли бы использовать это, чтобы увеличить прибыль. 

* Осьминог Пауль - настоящий прорицатель! Его предсказания всегда сбываются!

* Доходности по акциям Яндекса имеют нормальное распределение! 


Каждое из этих утверждений может быть проверено на данных. Сразу же стоит сказать важную вещь. __Гипотезы нельзя принмать!__ Если кто-то при вас сказал, что он принял гипотезу, нужно срочно ударить этого человека по губам. Предположим, что мы собрали данные, связанные с предсказаниями Пауля. Предположим, что данные не противоречат тому, что Пауль действительно видит будущее. Означает ли это, что Пауль - провидец? 

Вовсе нет, это означает, что текущие данные не противоречат этому. Возможно, если мы ещё понаблюдаем за предсказаниями Пауля, гипотеза о том, что он провидец, отвергнется. Если данные не противоречат нашему утверждению, это не является доказательством его верности. В матемтике да это всегда да, а нет это хотя бы один раз нет. Возможно, наше нет ещё не наступило. Именно поэтому гипотезы нельзя принимать, но можно не отвергать. __Ещё раз, ещё раз__ если кто-то говорит, что принял гипотезу, нужно уведомить его, что он её вовсе не принял, а не отверг. Не стесняйтесь делать это. 



## Примеры гипотез 





## Что делать с гипотезами

- Можно собирать данные и смотреть не противоречат ли им гипотезы

- Это не очень достоверно, надо быть готовым к ошибкам

- Гипотезы нельзя принмать, их не отвергают 




Давайте посмотрим на конкретный пример. Джеймс Бонд утверждает, что он предпочитает пить мартини взболтанным, но не смешанным. Чтобы проверить это на практике, можно предложить ему пройти так называемый blind test, или слепое тестирование. Можно было бы завязать ему глаза, несколько раз предложить на выбор взболтанный и смешанный мартини, а после этого спросить какой напиток он предпочитает В данном случае если бы Джеймс Бонд выбрал взболтанный напиток, это считалось бы успехом, потому что его выбор соответствовал бы его утверждению. В противном случае считалось бы, что произошла неудача. 

Сформулируем нашу первую гипотезу! Обычно, гипотезу, выдвинутую для проверки называют __нулевой или основной.__ 

$$H_0: \quad \text{ Джеймс Бонд не различает два вида напитков и выбирает наугад.}$$ 

На языке математики это утверждение можно сформулировать с посощью Бернулиевской случайной величины. Как мы помним, у неё есть единтвенный параметр $p$, вероятность успеха. Если Джеймс Бонд не различает два напитка и берёт их наугад, то $p = 0.5$. Если нет, то он должен выбирать правильный напиток чаще неправильного и $p > 0.5$. Нулевая гипотеза рвоно так и будет выглядеть:

$$
H_0: \quad p = \frac{1}{2}
$$

Эту гипотезу мы должны проверять против какой-то альтернативы. Альтернатива может быть разной. С одной стороны, мы можем рассмотреть двустороннюю альтернативу, состояшую в том, что агент Бонд различает два напитка между собой

$$
H_1: \quad p \ne \frac{1}{2}.
$$

С другой, мы могли бы рассмотреть одну из односторонних альтернатив. Например, что Бонд предпочитает именно вболтанный Мартини

$$
H_1: \quad p > \frac{1}{2}.
$$

Эксперимент с завязанными глазами нужно провести $n$ раз. В итоге у нас в руках окажется выборка, по которой мы сможем найти $\hat p$. Предположим, что на основе 10 испытаний мы получили, что $\hat p = \frac{6}{10}$. Означает ли это, что Бонд отвечает за свои слова? 

Возникает соблазн сказать, что да, так как число получилось больше $\frac{1}{2}$, и это ошибочный соблазн. Давайте вспомним немаловажный факт: наша оценка $\hat p$ это функция от выборки. При других наблюдениях мы могли бы получить другую оценку. Хочется понять насколько сильно эта оценка отличается от нашей.

Секундочку! Мы же уже умеем отвечать на этот вопрос. За точность оценки отвечает доверительный интервал. Мы помним, что 

$$\hat p \sim N \left(p, \frac{p(1-p)}{n} \right).$$

Исходя отсюда мы можем построить доверительный интервал и всё станет понятнее. Для нашей ситуации получим: 